In [2]:
import os
import io
from zipfile import ZipFile

In [3]:
import shutil
import csv

In [4]:
import re
import xapian
import pandas as pd
import numpy as np

In [5]:
pd.options.display.max_colwidth = 100

In [6]:
dbpath = "full_index"
search_results_path = "data/search_results.csv"

In [7]:
!xapian-delve $dbpath

UUID = b940719a-3fb5-4d57-8b01-3186790b1ade
number of documents = 3833466
average document length = 170.939
document length lower bound = 1
document length upper bound = 31154
highest document id ever used = 3833466
has positional information = true
revision = 392
currently open for writing = false


In [8]:
!xapian-delve -r 1833129 -d $dbpath

Data for record #1833129:

Term List for record #1833129: 17 2004 2007 3 QKim_Hyde Skim_hyde ZSkim_hyd Za Zand Zappear Zaustralian Zaway Zby Zcharact Zchris Zdepart Zfebruari Zfiction Zfirst Zfrom Zhe Zhemsworth Zhis Zhome Zhyde Zjonathan Zjuli Zkim Zkimber Zmade Zon Zopera Zplay Zscreen Zsoap Zthe Zwas a and appearance australian away by character chris departed february fictional first from he hemsworth his home hyde jonathan july kim kimberly made on opera played screen soap the was


# Retrieve documents relevant to claims

In [76]:
def get_doc_id(match):
    for term in match.document.termlist():
        term = term.term.decode("utf-8") 
        m = re.match("Q(.*)", term)
        if m:
            return m[1]
    return None

In [77]:
# Prepare enquiry object

# Open the database we're going to search.
db = xapian.Database(dbpath)

# Set up a QueryParser with a stemmer and suitable prefixes
queryparser = xapian.QueryParser()
queryparser.set_stemmer(xapian.Stem("en"))
queryparser.set_stemming_strategy(queryparser.STEM_SOME)

# Use an Enquire object on the database to run the query
enquire = xapian.Enquire(db)    

In [207]:
claim_df = pd.read_csv('data/claims.csv', index_col='claim_id')
claim_df.head()

,claim,source
claim_id,,
3,Chris Hemsworth appeared in A Perfect Getaway.,train
4,Chris Hemsworth disappeared in A Perfect Getaway.,train
6,Firefox is an application.,test
7,Roald Dahl is a writer.,train
8,Roald Dahl is a governor.,train


In [208]:
claim_df.shape

(165447, 2)

In [139]:
%%time
fields = ['claim_id', 'found_doc', 'rank', 'percentage', 'weight']
pagesize = 100
i = 0
results = []

with open(search_results_path, 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=fields)
    writer.writeheader()

    for claim_id, claim in claim_df.claim.items():
        if i % 100 == 0:
            print(i // 100, claim_id)
        i += 1

        query = queryparser.parse_query(claim)
        enquire.set_query(query)
        matches = enquire.get_mset(0, pagesize)

        query_results = []
        for match in matches:
            result = dict(
                claim_id = claim_id,
                found_doc = get_doc_id(match),
                rank = match.rank + 1,
                percentage = match.percent,
                weight = match.weight,            
            )
            query_results.append(result)
        writer.writerows(query_results)
        results += query_results
csvFile.close()

0 3
1 131
2 266
3 389
4 540
5 684
6 836
7 957
8 1102
9 1258
10 1395
11 1531
12 1670
13 1806
14 1934
15 2073
16 2205
17 2340
18 2492
19 2622
20 2767
21 2900
22 3027
23 3180
24 3319
25 3441
26 3590
27 3717
28 3853
29 3990
30 4127
31 4267
32 4406
33 4541
34 4678
35 4815
36 4952
37 5076
38 5218
39 5340
40 5469
41 5604
42 5741
43 5863
44 6018
45 6153
46 6282
47 6404
48 6532
49 6655
50 6788
51 6908
52 7036
53 7166
54 7299
55 7419
56 7565
57 7684
58 7813
59 7943
60 8072
61 8210
62 8346
63 8479
64 8616
65 8759
66 8899
67 9028
68 9148
69 9297
70 9430
71 9577
72 9718
73 9851
74 9978
75 10116
76 10270
77 10400
78 10534
79 10667
80 10793
81 10935
82 11080
83 11220
84 11348
85 11505
86 11635
87 11766
88 11901
89 12039
90 12165
91 12301
92 12440
93 12576
94 12707
95 12837
96 12967
97 13102
98 13238
99 13380
100 13521
101 13652
102 13775
103 13917
104 14049
105 14185
106 14325
107 14464
108 14603
109 14733
110 14864
111 14991
112 15125
113 15268
114 15390
115 15526
116 15664
117 15796
118 15927
119 1

830 113236
831 113371
832 113516
833 113664
834 113803
835 113941
836 114103
837 114235
838 114381
839 114508
840 114640
841 114783
842 114927
843 115068
844 115210
845 115358
846 115511
847 115641
848 115780
849 115922
850 116056
851 116220
852 116380
853 116525
854 116674
855 116804
856 116928
857 117063
858 117219
859 117357
860 117481
861 117632
862 117783
863 117916
864 118054
865 118194
866 118333
867 118472
868 118611
869 118740
870 118876
871 119013
872 119146
873 119283
874 119424
875 119559
876 119689
877 119846
878 119989
879 120121
880 120283
881 120429
882 120574
883 120711
884 120846
885 120990
886 121127
887 121276
888 121417
889 121561
890 121696
891 121838
892 121980
893 122124
894 122266
895 122407
896 122551
897 122688
898 122826
899 122967
900 123114
901 123249
902 123384
903 123528
904 123683
905 123826
906 123964
907 124107
908 124243
909 124371
910 124499
911 124629
912 124756
913 124885
914 125020
915 125160
916 125289
917 125437
918 125578
919 125719
920 125853

1527 211816
1528 211960
1529 212122
1530 212260
1531 212407
1532 212514
1533 212639
1534 212792
1535 212921
1536 213058
1537 213187
1538 213290
1539 213399
1540 213548
1541 213690
1542 213862
1543 213976
1544 214082
1545 214208
1546 214328
1547 214436
1548 214544
1549 214742
1550 214906
1551 215037
1552 215184
1553 215359
1554 215494
1555 215681
1556 215800
1557 215944
1558 216055
1559 216188
1560 216317
1561 216478
1562 216622
1563 216754
1564 216871
1565 217033
1566 217186
1567 217334
1568 217461
1569 217638
1570 217755
1571 217938
1572 218060
1573 218188
1574 218311
1575 218420
1576 218536
1577 218641
1578 218759
1579 218869
1580 218978
1581 219124
1582 219268
1583 219449
1584 219559
1585 219672
1586 219891
1587 220031
1588 220201
1589 220367
1590 220546
1591 220712
1592 220852
1593 220974
1594 221103
1595 221322
1596 221435
1597 221567
1598 221709
1599 221812
1600 221914
1601 222030
1602 222171
1603 222336
1604 222452
1605 222576
1606 222738
1607 222881
1608 223002
1609 223122
1610

# Analyse Found Documents

## Read search results

In [9]:
results_df = pd.read_csv(search_results_path)

In [10]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16544700 entries, 0 to 16544699
Data columns (total 5 columns):
claim_id      int64
found_doc     object
rank          int64
percentage    int64
weight        float64
dtypes: float64(1), int64(3), object(1)
memory usage: 631.1+ MB


In [11]:
#results_df.sort_values('percentage', ascending=False).head(100)
results_df.sort_values('weight', ascending=False).head(10)

,claim_id,found_doc,rank,percentage,weight
12993200,179614,Michael_Giacchino,1,100,246.832666
12993400,179617,Michael_Giacchino,1,100,246.832666
12994100,179627,Michael_Giacchino,1,100,246.832666
12996200,179663,Michael_Giacchino,1,100,246.832666
14294200,199027,International_relations,1,98,236.997039
14300000,199103,International_relations,1,98,236.997039
14293700,199019,International_relations,1,98,236.997039
14297000,199061,International_relations,1,98,236.997039
14297300,199065,International_relations,1,98,236.997039
14294500,199030,International_relations,1,98,236.997039


In [12]:
func = lambda x: set(x)
found_docs_df = results_df.pivot_table(index='claim_id', values='found_doc', aggfunc=func)
found_docs_df.head()

,found_doc
claim_id,
3,"{Grimethorpe, Luke_Hemsworth, Chris_Joris, Team_Soho, The_Perfect_Assistant, Perfect_10, Chris_H..."
4,"{Grimethorpe, Aquil_Abdullah, Luke_Hemsworth, Gone_Maggie_Gone, Team_Soho, Arthur_Bunting, The_P..."
6,"{BlueGriffon, SeaMonkey, Xprint, Asa_Dotzler, Motion_JPEG, Secure_cookies, XPInstall, FireFTP, W..."
7,"{Wonka, The_Magic_Finger, David_Coke, Charlie_and_the_Chocolate_Factory_video_games, Edward_the_..."
8,"{Wonka, The_Magic_Finger, David_Coke, Charlie_and_the_Chocolate_Factory_video_games, Edward_the_..."


## Read claims

In [25]:
labelled_claims_df = pd.read_json('data/labelled_claims.json').sort_index()
#mask = labelled_claims_df.source == 'dev'
#labelled_claims_df = labelled_claims_df[mask]

In [26]:
labelled_claims_df = labelled_claims_df.join(found_docs_df)

In [27]:
func = lambda x: set(x['evidence_docs']).difference(x['found_doc'])
labelled_claims_df['missed_docs'] = labelled_claims_df.apply(func, axis=1)
labelled_claims_df['missed_count'] = labelled_claims_df['missed_docs'].apply(len)

In [28]:
labelled_claims_df.head()

,claim,source,evidence_docs,label,found_doc,missed_docs,missed_count
3,Chris Hemsworth appeared in A Perfect Getaway.,train,[Chris_Hemsworth],SUPPORTS,"{Tom_Devanney, Conditional_perfect, Markov_perfect_equilibrium, Chris_McMenamin, The_Perfect_Sco...",{},0
4,Chris Hemsworth disappeared in A Perfect Getaway.,train,[],NOT ENOUGH INFO,"{Tom_Devanney, Arthur_Bunting, Hard_Rock_Treasures, The_Perfect_Score, Stabilized_images, Neil_R...",{},0
7,Roald Dahl is a writer.,train,[Roald_Dahl],SUPPORTS,"{Esio_Trot, Sophie_Dahl, Roald_Dahl, Piker, The_Best_of_Roald_Dahl, The_Wonderful_Story_of_Henry...",{},0
8,Roald Dahl is a governor.,train,[Roald_Dahl],REFUTES,"{Esio_Trot, Sophie_Dahl, Piker, Roald_Dahl, The_Best_of_Roald_Dahl, The_Twits, The_Wonderful_Sto...",{},0
9,Ireland has relatively low-lying mountains.,train,[Ireland],SUPPORTS,"{List_of_highest_mountains_of_Tasmania, Rockefeller_Mountains, Ireland, Croghan_Hill, Pandemoniu...",{},0


In [40]:
recall_df = labelled_claims_df.pivot_table(index='missed_count', columns='source',# margins=True,
                                                values='claim', aggfunc='count', fill_value=0)
recall_df.head()

source,dev,train
missed_count,,
0,3837,105020
1,1005,34150
2,111,4146
3,19,903
4,18,469


In [42]:
(recall_df / recall_df.sum() * 100).head(1)

source,dev,train
missed_count,,
0,76.724655,72.204003


## Evidence found

In [43]:
found_df = labelled_claims_df[(labelled_claims_df.missed_count == 0) & (labelled_claims_df.label != 'NOT ENOUGH INFO')]

In [44]:
found_df.evidence_docs.astype(str).value_counts().to_frame()

,evidence_docs
['Snoop_Dogg'],166
['Wyatt_Earp'],148
['Marlon_Brando'],134
['Miley_Cyrus'],120
['Adele'],118
['David_Beckham'],102
['Tim_Rice'],102
['Anne_Hathaway'],100
['Carrie_Fisher'],98
['Lady_Gaga'],94


## Evidence missed

In [45]:
missed_df = labelled_claims_df[labelled_claims_df.missed_count > 0]

In [46]:
missed_df.missed_docs.astype(str).value_counts().to_frame()

,missed_docs
{'Deadpool_-LRB-film-RRB-'},118
{'Planet_of_the_Apes_-LRB-1968_film-RRB-'},114
{'The_Prestige_-LRB-film-RRB-'},103
{'Glee_-LRB-TV_series-RRB-'},102
{'François_de_Belleforest'},100
{'AC/DC'},97
{'Eagles_-LRB-band-RRB-'},95
{'United_States'},94
{'Beyoncé'},93
{'O._J._Simpson'},90
